# JOUR7280/COMM7780 Big Data Analytics for Media and Communication
# Tutorial: Pandas Example

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movie_df = pd.read_csv('../data/' + 'movie_metadata.csv')
movie_df.head()

**Example 1** Read the 'movie_metadata.csv' file. Tell:

    a) how many tuples and fields(columns) the dataset has.
    b) what are the columns

In [ ]:
movie_shape = movie_df.shape
numTuples = movie_shape[0]
numFields = movie_shape[1]
print('{} tuples and {} columns \n'.format(numTuples, numFields)) # {} is the placeholder

movie_col_name = list(movie_df.columns)
print(movie_col_name)

In [ ]:
movie_df.columns

## Handling missing values 
**Example 2** 

In [ ]:
movie_df

For example, in row 5039, the director_name is NaN. [`dropna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method will delete the rows with missing data.

Please go through the following process to handle missing values
    
    a) Find out the sum of missing values for each column
    b) Drop all rows containing any missing value
    
Tell how many tuples are dropped. Explain whether it is a good way to handle missing value for the case? 
(please print out your answer)

In [ ]:
# sum up NAs
print(movie_df.isna().sum())

In [ ]:
# return a copy of cleaned df
clean_movie_df = movie_df.dropna()

num_dropped = numTuples - clean_movie_df.shape[0]

print(num_dropped, 'tuples dropped.')

This is not a good way to handle missing values since too much information gets lost.

**Example 3** Try the following methods to handle missing values inplace.

    a) replace missing values in 'director_name'with 'unknown'
    
   The first argument of `fillna` is a `dictionary`, to indicate which value to use for specified column(s).

In [ ]:
# replace with unknown
movie_df.fillna({'director_name': 'unknown'}, inplace=True)
movie_df

    b) use mean to fill in missing values in'gross' and 'budget'
    c) use median to fill in missing values in 'num_critic_for_reviews'and 'num_user_for_reviews'
    d) use mode to fill in missing values in 'country'
    
Recall from math: 
* The "mean" is the "average": you add up all the numbers and then divide by the number of numbers.
* The "median" is the "middle" value in the list of numbers.
* The "mode" is the value that occurs most often. If no number in the list is repeated, then there is no mode for the list.

In [ ]:
# replace with mean
mean_budget = movie_df['budget'].mean()
mean_gross = movie_df['gross'].mean()

movie_df.fillna({'budget': mean_budget, 'gross': mean_gross}, inplace=True)

# replace with median
median_critic = movie_df['num_critic_for_reviews'].median()
median_user = movie_df['num_user_for_reviews'].median()

movie_df.fillna({'num_critic_for_reviews': median_critic,'num_user_for_reviews': median_user}, inplace=True)

# replace with mode
mode_country = movie_df['country'].mode()
movie_df.fillna({'country': mode_country[0]}, inplace=True)

    e) replace missing values in 'duration' with the last observation

In [ ]:
# replace with the last observation
movie_df['duration'].fillna(method='ffill', inplace=True)

`ffill`: propagate last valid observation forward

See more about `fillna` [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)

**Example 4** There are some duplicated rows in the dataset.

    a) tell how many duplicated rows are in the dataset
    
   `dataframe.duplicated()` returns boolean Series denoting duplicate rows.

In [ ]:
movie_df.duplicated()

In [ ]:
# number of duplications
duplicate_df = movie_df[movie_df.duplicated()]
print('# of duplicated rows:', duplicate_df.shape[0])

    b) remove duplicates
    c) Tell how many rows left after removing duplicates

In [ ]:
# remove duplicates
movie_df.drop_duplicates(inplace=True)

# how many left?
print(movie_df.shape[0], 'rows left after removing duplicates')

**Example 5** Create a new column named 'imdb_label'. The lable is given based on the following criteria:

    a) imdb_score in (0, 4], label 'bad'
    b) imdb_score in (4, 7], label 'moderate'
    c) imdb_score in (7, 10], label 'good'
You should consider **data binning** to finish the task.

`pd.cut` method bins values into discrete intervals.
- `labels`: Specifies the labels for the returned bins. 
- `bins`: The criteria to bin by.

In [ ]:
# define lables and categories
labels_list = ['bad', 'moderate', 'good']
category = [0., 4., 7., 10.]

# create the news column
movie_df['imdb_label'] = pd.cut(movie_df['imdb_score'], labels=labels_list, bins=category)

movie_df.head()

**Example 6** Try to get the following info from the data:

    a) basic statistical characteristics of each numerical feature

In [ ]:
# numerical summary
movie_df.describe()

By default, the result of `dataframe.describe()` will include all numeric columns.

    b) basic statistical characteristics of each non-numerical feature

In [ ]:
# non-numerical summary
movie_df.describe(include=['object', 'category'])

By setting parameter `include=['object', 'category']`, it limits the result to object and categorical columns.

    c) the distributions(normalized) of imdb_label

In [ ]:
# distribution
movie_df['imdb_label'].value_counts(normalize = True)

`value_counts` return a Series containing counts of unique values.
- normalize = True: the object returned will contain the relative frequencies of the unique values.

**Example 7** Sort the dataset in following two ways:

    a) by budget in ascending order
    b) by budget and imdb_score in descending order   

In [ ]:
# budget in ascending order
movie_df.sort_values(by='budget').head()

In [ ]:
# budget and imdb_score in descending order
movie_df.sort_values(by=['imdb_score', 'budget'], ascending=[False, False]).head()

**Example 8** Practice the following steps:

    a) Group the dataset by 'imdb_label'
    b) Select 'gross', 'budget' and 'imdb_score' to display
    c) Apply aggregation functions to the selected columns, including mean, std, min, max
    

In [ ]:
# define columns to show
columns_to_show = ['gross', 'budget', 'imdb_score']

# performing grouping and apply functions to groups
movie_df.groupby(['imdb_label'])[columns_to_show].agg([np.mean, np.std, np.min, np.max])

In [ ]:
movie_df.groupby(['imdb_label'])

**Example 9** Split the 'imdb_label' column into three numerical columns: 

    a)'label_bad'
    b)'label_morderate'
    c)'label_good'. 
Use 1 to indicated the correct label and 0 inversely. Write this new dataframe to 'movie_metadata_new.csv'

We first convert categorical variables ('imdb_label' column) into dummy/indicator variables. A **dummy variable** (aka, an indicator variable) is a numeric variable that represents categorical data, i.e., takes only the value 0 or 1 to indicate the absence or presence.

In [ ]:
# convert categorical variables into dummy/indicator variables
dfDummies = pd.get_dummies(movie_df['imdb_label'], prefix='label')
dfDummies.head()

`pd.get_dummies` converts categorical variable into dummy/indicator variables.
- prefix='label': append word 'label' to column names.

Then we concatenate `dfDummies` with the original dataframe and save to file.

In [ ]:
# concatenate with the original dataframe
encoded_movie_df = pd.concat([movie_df, dfDummies], axis=1)

# write to new csv file
encoded_movie_df.to_csv('../data/' + 'movie_metadata_new.csv')

encoded_movie_df.head()